In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymongo import MongoClient
import configparser
import logging
import logging.config
sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
os.environ['INIDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/ini'
os.environ['LOGDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/log'
INIDIR=os.environ['INIDIR'] 
LOGDIR=os.environ['LOGDIR'] 

logging.config.fileConfig(os.path.join(INIDIR,'logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")


In [3]:
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(INIDIR,'db_config.ini'), encoding='utf-8')

from src.mongo_handler import MongoHandler

In [4]:

import  kuloko_handler.handler.socket_handler as skt_api

private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(INIDIR,'private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(INIDIR,'config.ini'), encoding='utf-8')


['/Users/macico/Dropbox/kaggle/kuloko/ini/config.ini']

In [5]:
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")

In [6]:
# mongo_db.db['orderbook'].insert_one({"test":"rrr"})

In [7]:
for db_data in mongo_db.db['orderbook'].find():
    print(db_data)

In [8]:
mongo_db.db['orderbook'].find_one()

In [9]:
mongo_db.delete_all()


In [9]:
orderbooks = skt_api.Orderbooks(logger, general_config_ini,private_api_ini)
orderbooks.connect(orderbooks.get_url(),'BTC')
orderbooks.subscribe()
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")

In [7]:
import time
retry_cnt=0
max_retry_cnt=60
while True:
    try:
        data = orderbooks.get()
        if len(data)==0:
            continue
        insert_json =[ orderbooks.convert_shape(_data,5,"json") for _data in data ]
        mongo_db.insert_many(insert_json)
        time.sleep(5)
    except Exception as e:
        if retry_cnt < max_retry_cnt:
            continue
        print("DONE",e)
        break


--- request header ---
GET /ws/public/v1 HTTP/1.1
Upgrade: websocket
Host: api.coin.z.com
Origin: http://api.coin.z.com
Sec-WebSocket-Key: lLzfc3L8fSUWHSXXD97opw==
Sec-WebSocket-Version: 13
Connection: upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Thu, 03 Dec 2020 22:22:41 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: aayvBySahyDt+vE3sVDqZVM4Mu4=
X-Cache: Miss from cloudfront
Via: 1.1 53d9d56dd0a523b5894842ad66a68756.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: NRT57-C2
X-Amz-Cf-Id: l9yvzPhPEVO4kvCAszcwn0fn5ANnPXdsjJS5nJqrqN1ycAwhIMbWFQ==
-----------------------
send: b'\x81\xc2\x0cV\xda\xe2wt\xb9\x8da;\xbb\x8cht\xe0\xc2.%\xaf\x80\x7f5\xa8\x8bn3\xf8\xce,t\xb9\x8am8\xb4\x87`t\xe0\xc2.9\xa8\x86i$\xb8\x8dc=\xa9\xc0 v\xf8\x91u;\xb8\x8d`t\xe0\xc2.\x14\x8e\xa1.+'
send: b'\x8a\x804\xb1\x9f\x7f'
send: b'\x8a\x80l\xa6*\x7f'
send: b'\x8a\x80\xdcaqC'
send: b'\x8a\x80\xa8\xeb\xe2\xa1'
send: b'\x8a\x80i\x88-N'
send: b'\x8a\x80\xf9`\

KeyboardInterrupt: 

In [10]:
orderbooks.disconnect()

send: b'\x88\x82\xbc\xbb\xb8S\xbfS'
error from callback <bound method Socket.on_close of <kuloko_handler.handler.socket_handler.Orderbooks object at 0x1169a2590>>: Connection is already closed.
  File "/Users/macico/.pyenv/versions/py37b/lib/python3.7/site-packages/websocket/_app.py", line 344, in _callback
    callback(*args)
  File "../kuloko_handler/handler/socket_handler.py", line 134, in on_close
    self.ws.send(json.dumps(message))
  File "/Users/macico/.pyenv/versions/py37b/lib/python3.7/site-packages/websocket/_app.py", line 156, in send
    "Connection is already closed.")


In [ ]:
orderbooks.disconnect()

In [17]:
data = orderbooks.get()

In [18]:
data

[]